In [1]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

# 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import env

In [3]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = """SELECT * FROM logs LEFT JOIN cohorts ON logs.user_id = cohorts.id"""
df = pd.read_sql(query, url)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0


In [4]:
df.dtypes

date           object
time           object
path           object
user_id         int64
cohort_id     float64
ip             object
id            float64
name           object
slack          object
start_date     object
end_date       object
created_at     object
updated_at     object
deleted_at     object
program_id    float64
dtype: object

In [7]:
df.shape

(900223, 15)

In [5]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

In [15]:
df.rename(columns = {'path':'endpoint'}, inplace = True)

In [16]:
pages = df['endpoint'].resample('d').count()
pages.head()

date
2018-01-26    572
2018-01-27    230
2018-01-28    170
2018-01-29    830
2018-01-30    652
Freq: D, Name: endpoint, dtype: int64

In [6]:
df.dtypes

date          datetime64[ns]
time                  object
path                  object
user_id                int64
cohort_id            float64
ip                    object
id                   float64
name                  object
slack                 object
start_date            object
end_date              object
created_at            object
updated_at            object
deleted_at            object
program_id           float64
dtype: object

In [25]:
df.groupby('program_id').name.value_counts()

program_id  name                                  
1.0         Ike                                       17913
            Arches                                     7404
            Glacier                                    1803
            Everglades                                 1701
            Carlsbad                                   1562
            Badlands                                   1541
            Quincy                                     1533
            Franklin                                   1408
            Joshua                                      889
            Denali                                      692
            Lassen                                      596
            Hampton                                     483
            Olympic                                     175
            Kings                                        94
2.0         Fortuna                                   12329
            Redwood                              

In [8]:
def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['endpoint'].resample('d').count()
    return pages_one_user

In [9]:
def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

In [10]:
def plot_bands(my_df, user):
    '''
    This function plots the bolliger bands of the page views for a single user
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

In [12]:
def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

In [19]:
# test on a single user

user = 5
span = 30
weight = 3

# initialize empty df
anomalies = pd.DataFrame()

# call function to create a new df with anomalies for the user 
user_df = find_anomalies(df, user, span, weight)

# concatenate the above df created with the empty anomalies df
anomalies = pd.concat([anomalies, user_df], axis=0)

In [20]:
anomalies

,pages_one_user,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-07-09,16,1.463733,14.847740,-11.920274,1.043046,5
2018-07-19,38,3.202948,32.767157,-26.361262,1.088500,5


In [21]:
user_df

,pages_one_user,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-07-09,16,1.463733,14.847740,-11.920274,1.043046,5
2018-07-19,38,3.202948,32.767157,-26.361262,1.088500,5


In [22]:

# loop through all users to find anomalies for each
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [23]:
anomalies

,pages_one_user,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-01-29,44,3.623334,42.081344,-34.834677,1.024945,1
2019-07-08,101,8.248768,96.507472,-80.009936,1.025451,1
2018-10-15,9,0.581421,8.457414,-7.294571,1.034446,3
2019-01-09,4,0.262470,3.802115,-3.277175,1.027953,3
2019-04-06,2,0.129825,1.889712,-1.630061,1.031334,3
...,...,...,...,...,...,...
2021-01-24,7,0.574393,6.857226,-5.708440,1.011362,817
2021-03-25,59,6.824556,58.415342,-44.766230,1.005666,843
2021-02-23,67,8.229409,66.322904,-49.864086,1.005828,851
